有 `CODE LIST` 标识的为书中例子。

为了能够理解书中例子，一些（Preparation）模块记录了必要的知识

# (Preparation) 模板 (对应1.1.2)

ステップ：<br>
用 template() 方法创作模板；<br>
用 .substitude() 直接替换，或用 {} 创建对应的字典，用 ** 传入字典的值

补充：<br>
单星号 * 用于对列表LIST或元组tuple中的元素进行取出<br>
双星号 ** 可将字典里的“值”取出

In [2]:
# example
from string import Template
 
t = Template('$name is the $job of $company')
s = t.substitute(name='Tim Cook', job='CEO', company='Apple Inc.')
print(type(t))
print(s)
 
# dictionary as substitute argument
d = {"name": "Tim Cook", "job": "CEO", "company": "Apple Inc."}
s = t.substitute(**d)
print(s)

<class 'string.Template'>
Tim Cook is the CEO of Apple Inc.
Tim Cook is the CEO of Apple Inc.


In [ ]:
# 补充说明
import numpy as np
LIST = [3, 6]
print(np.arange(*LIST))
def parrot(voltage, state='a stiff', action='voom'):
    print ("-- This parrot wouldn't", action, "if you put", voltage, 
    "volts through it.", "E's", state, "!")

d = {"voltage": "four million", "state": "bleedin' demised", "action": "VOOM"}
parrot(**d) # **d 取出 values, 并根据 keys 送入对应参数
print(*d) # *d 会取出 dictionary{keys:values}中的 keys。

[3 4 5]
-- This parrot wouldn't VOOM if you put four million volts through it. E's bleedin' demised !
voltage state action


Template对象有两个方法：**substitute()、safe_substitute()**。<br>

substitute()更为严谨，在key缺少的情况下会报一个KeyError的异常。<br>
safe_substitute()在缺少key的情况下，直接原封不动的把字符串显示出来。

In [ ]:
# safe_substitude() 方法
from string import Template
 
t = Template('$name is the $job of $company')
 
s = t.safe_substitute(name='Tim Cook', job='CEO')
print(s)

Tim Cook is the CEO of $company


**打印模板字符串**

模板对象具有返回模板字符串的“ template”属性。

**转义$符号** 

我们可以使用$$来转义$符号并将其视为普通字符串的一部分。

In [ ]:
t = Template('$name is the $job of $company')
print('Template String =', t.template)

t = Template('$$ is called $name')
s = t.substitute(name='Dollar')
print(s)

Template String = $name is the $job of $company
$ is called Dollar


**$ {identifier}示例**

$ 用来提示后面为替换的变量，同时可用{}来区分变量和周围的文本

In [ ]:
t = Template('$noun adjective is ${noun}ing')
s = t.substitute(noun='Test')
print(s)

Test adjective is Testing


# 1.1 string: 文字常量和模板

## 1.1.1 函数

In [1]:
# CODE LIST 1-1
# capwords(): 将字符串中所有单词首字母大写
import string

s = 'The quick brown fox jumped over the lazy dog.'

print(s)
print(string.capwords(s))

The quick brown fox jumped over the lazy dog.
The Quick Brown Fox Jumped Over The Lazy Dog.


## 1.1.2 **模板**

Python String Template类用于创建简单的模板字符串，以后可以在其中替换字段以创建字符串对象。

1-2 提供了三个例子：简单模板、%操作符(类似字符串拼接)、str.format()的新格式化字符串语法

注1：`escape`是`转义`的意思
> In the first two cases, the trigger character ($ or %) is escaped by repeating it twice. <br>
> For the format syntax, both { and } need to be escaped by repeating them.

In [2]:
# CODE LIST 1-2
import string

values = {'var': 'foo'}

# """ ... """ ：多行注释；或定义多行字符串
t = string.Template("""
Variable        : $var
Escape          : $$
Variable in text: ${var}iable
""")

print('TEMPLATE:', t.substitute(values))

s = """
Variable        : %(var)s
Escape          : %%
Variable in text: %(var)siable
"""

print('INTERPOLATION:', s % values)

s = """
Variable        : {var}
Escape          : {{}}
Variable in text: {var}iable
"""

print('FORMAT:', s.format(**values))

TEMPLATE: 
Variable        : foo
Escape          : $
Variable in text: fooiable

INTERPOLATION: 
Variable        : foo
Escape          : %
Variable in text: fooiable

FORMAT: 
Variable        : foo
Escape          : {}
Variable in text: fooiable



缺少输入值时，substitute()方法会产生KeyError

注：
    `try ... except Exception as e`
中， e 是前面 Exception 类的一个实例

In [15]:
# CODE LIST 1-3
# substitute 和 safe_substitute()
import string

values = {'var': 'foo'}

t = string.Template("$var is here but $missing is not provided")

try:
    print('substitute()     :', t.substitute(values))
except KeyError as err:
    print('ERROR:', str(err))

print('safe_substitute():', t.safe_substitute(values))

ERROR: 'missing'
safe_substitute(): foo is here but $missing is not provided


## 1.1.3 高级模板（学完正则表达式再来弄）

可以更改变量定义符号$，以及利用正则式替换变量名，以改变默认语法

`delimiter` : 定义符号<br>
`underscore`: 下划线

In [1]:
# CODE LIST 1-4
# 本例中，自定义了 % 以替换原有的变量定义符号 $
import string

class MyTemplate(string.Template):
    delimiter = '%'
    idpattern = '[a-z]+_[a-z]+'


template_text = '''
  Delimiter : %%
  Replaced  : %with_underscore
  Ignored   : %notunderscored
'''

d = {
    'with_underscore': 'replaced',
    'notunderscored': 'not replaced',
}

t = MyTemplate(template_text)
print('Modified ID pattern:')
print(t.safe_substitute(d))

<class 'dict'>
Modified ID pattern:

  Delimiter : %
  Replaced  : replaced
  Ignored   : %notunderscored



更复杂的修改：覆盖pattern属性并定义一个全新的正则表达式。

所提供的模式必须包含4个命名组，分别捕获 转义定界符、命名变量、加括号的变量名、不合法的定界符模式

In [3]:
# CODE LIST 1-5
import string

t = string.Template('$var')
print(t.pattern.pattern)


    \$(?:
      (?P<escaped>\$) |   # Escape sequence of two delimiters
      (?P<named>(?a:[_a-z][_a-z0-9]*))      |   # delimiter and a Python identifier
      {(?P<braced>(?a:[_a-z][_a-z0-9]*))}  |   # delimiter and a braced identifier
      (?P<invalid>)              # Other ill-formed delimiter exprs
    )
    


新模式举例（暂时没去理解）

In [4]:
import re
import string


class MyTemplate(string.Template):
    delimiter = '{{'
    pattern = r'''
    \{\{(?:
    (?P<escaped>\{\{)|
    (?P<named>[_a-z][_a-z0-9]*)\}\}|
    (?P<braced>[_a-z][_a-z0-9]*)\}\}|
    (?P<invalid>)
    )
    '''


t = MyTemplate('''
{{{{
{{var}}
''')

print('MATCHES:', t.pattern.findall(t.template))
print('SUBSTITUTED:', t.safe_substitute(var='replacement'))

MATCHES: [('{{', '', '', ''), ('', 'var', '', '')]
SUBSTITUTED: 
{{
replacement



## 1.1.4 Formatter

## 1.1.5 常量

In [17]:
# CODE LIST 1-7
# 打印了 string 模块中包含的大量与 ASCII 和数值字符集相关的常量
import inspect
import string


def is_str(value):
    return isinstance(value, str)


for name, value in inspect.getmembers(string, is_str):
    if name.startswith('_'):
        continue
    print('%s=%r\n' % (name, value))

ascii_letters='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

ascii_lowercase='abcdefghijklmnopqrstuvwxyz'

ascii_uppercase='ABCDEFGHIJKLMNOPQRSTUVWXYZ'

digits='0123456789'

hexdigits='0123456789abcdefABCDEF'

octdigits='01234567'

printable='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

punctuation='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

whitespace=' \t\n\r\x0b\x0c'



# 練習

尝试使用正则表达式构造模板

In [12]:
from string import Template
class newTemplate(Template):
    delimiter = '$'
    idpattern = '[a-z]+_[a-z]+'

template_text = '''
    $Kirisame_Marisa where $Hakurei
'''
d = {
    'ai_ueo' : 'Remilia_Scarlet',
    'Kirisame_Marisa' : 'Remilia_Scarlet',
    #'kakiku' : 'Izayoi',
}

t = newTemplate(template_text)
s = t.safe_substitute(d)

print(s)
 



    $Kirisame_Marisa where $Hakurei

